##  Elastic net Project: Predicting car Prices 
 * Shiraz Hemo - 207487406
 * Daniel Yesharim-212357776
 * LINK TO GITHUB- https://github.com/DanielYesharim/dataPython

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

##  Call Data 

In [4]:
df = pd.read_csv('dataset.csv')

In [5]:
df.head()

,manufactor,Year,model,Hand,Gear,capacity_Engine,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test,Supply_score
0,יונדאי,2015,i35,2,אוטומטית,1600,בנזין,פרטית,פרטית,רעננה - כפר סבא,רעננה,51000.0,2.0,11/07/2023,11/07/2023,['רכב שמור בקנאות\nמוכרת עקב קבלת רכב חברה'],כחול כהה מטאלי,144000,NaN,NaN
1,ניסאן,2018,ניסאן מיקרה,1,אוטומטית,1200,בנזין,פרטית,פרטית,מושבים בשרון,אבן יהודה,49000.0,0.0,06/04/2022,22/05/2022,['שמורה כל התוספות'],כחול בהיר,69000,NaN,NaN
2,סוזוקי,2010,סוזוקי סוויפט,1,אוטומטית,1450,בנזין,NaN,NaN,רמת,רמת,22500.0,1.0,29/10/2022,29/10/2022,['רכב במצב מתוחזק ברמה גבוהה טסט עד אפריל 2023'],NaN,145000,NaN,NaN
3,טויוטה,2016,אוריס,1,טיפטרוניק,1600,בנזין,פרטית,פרטית,נס ציונה - רחובות,רחובות,63000.0,5.0,16/05/2024,16/05/2024,['אוטו במצב חדש!! שמור בקנאות!! נהג יחיד מטופל...,אפור מטאלי,27300,NaN,NaN
4,קיה,2012,פיקנטו,1,אוטומטית,1248,בנזין,NaN,NaN,"ראשל""צ והסביבה",ראשון לציון,37000.0,1.0,13/06/2022,13/06/2022,['שמור'],NaN,70000,NaN,4.0


## Exploratory Data Analysis

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   manufactor       1500 non-null   object 
 1   Year             1500 non-null   int64  
 2   model            1500 non-null   object 
 3   Hand             1500 non-null   int64  
 4   Gear             1499 non-null   object 
 5   capacity_Engine  1474 non-null   object 
 6   Engine_type      1495 non-null   object 
 7   Prev_ownership   771 non-null    object 
 8   Curr_ownership   772 non-null    object 
 9   Area             1351 non-null   object 
 10  City             1500 non-null   object 
 11  Price            1500 non-null   float64
 12  Pic_num          1476 non-null   float64
 13  Cre_date         1500 non-null   object 
 14  Repub_date       1500 non-null   object 
 15  Description      1500 non-null   object 
 16  Color            730 non-null    object 
 17  Km            

In [7]:
df.describe() # get some statistics on numeric columns

,Year,Hand,Price,Pic_num,Supply_score
count,1500.000000,1500.000000,1500.000000,1476.000000,439.000000
mean,2014.346000,2.349333,51085.086000,2.836721,1581.011390
std,3.815406,1.229217,21933.308735,3.499312,2651.323149
min,1983.000000,1.000000,18200.000000,0.000000,0.000000
25%,2012.000000,1.000000,32000.000000,1.000000,20.000000
50%,2015.000000,2.000000,48000.000000,1.000000,474.000000
75%,2017.000000,3.000000,68000.000000,4.000000,2402.000000
max,2023.000000,10.000000,99960.000000,40.000000,16508.000000


In [8]:
df.dtypes  

manufactor          object
Year                 int64
model               object
Hand                 int64
Gear                object
capacity_Engine     object
Engine_type         object
Prev_ownership      object
Curr_ownership      object
Area                object
City                object
Price              float64
Pic_num            float64
Cre_date            object
Repub_date          object
Description         object
Color               object
Km                  object
Test                object
Supply_score       float64
dtype: object

In [9]:
## we can see that the columns - capacity_Engine and Km are of type object, so we will change them into a int64.

In [10]:
df.isna().sum() #all the missing valus in etch coulmes

manufactor            0
Year                  0
model                 0
Hand                  0
Gear                  1
capacity_Engine      26
Engine_type           5
Prev_ownership      729
Curr_ownership      728
Area                149
City                  0
Price                 0
Pic_num              24
Cre_date              0
Repub_date            0
Description           0
Color               770
Km                   89
Test               1368
Supply_score       1061
dtype: int64

In [11]:
## we will remove the data with large number of null.

In [12]:
df.capacity_Engine.value_counts().head(20)

capacity_Engine
1600     227
2000     180
1400     138
1200     135
1800     113
1500      92
1000      65
1,400     46
1250      35
1300      34
1248      33
1,200     32
2500      25
1,800     19
1,600     14
4500      12
1,500     10
2,000      9
1598       9
1798       9
Name: count, dtype: int64

In [13]:
## we can see that some values contain , in numbers. 

In [14]:
df.Km.value_counts()

Km
180000     27
200000     26
90000      24
130000     23
170000     21
           ..
195100      1
330000      1
148600      1
121,200     1
163899      1
Name: count, Length: 470, dtype: int64

In [15]:
## we can see that some values contain , in numbers. 

In [16]:
df.manufactor.value_counts()

manufactor
מאזדה          172
קיה            167
יונדאי         135
פולקסווגן      128
סקודה          120
הונדה          119
טויוטה          96
סוזוקי          65
אאודי           64
שברולט          61
אופל            42
מיצובישי        40
סיטרואן         38
סובארו          37
ניסאן           32
ב.מ.וו          32
רנו             28
מרצדס           24
פיג'ו           20
פורד            18
וולוו           13
דייהטסו         11
מיני             9
Lexsus           8
אלפא רומיאו      8
לקסוס            7
קרייזלר          6
Name: count, dtype: int64

In [17]:
## we can see that there is Lexsus type and 'לקסוס' type.  

In [18]:
df.Engine_type.value_counts()

Engine_type
בנזין         1346
דיזל            51
היבריד          35
היברידי         27
גז              25
טורבו דיזל       8
חשמלי            3
Name: count, dtype: int64

In [19]:
## same as manufactor. we will combine 'היבריד' coulom and היברידי colom.

## Prepare data (feature engineering)
###  נבצע הכנה של הנתונים לטובת מודל חיזוי המחיר

* נשנה את הערכים הלא מספריים בעמודות קילומטר ונפח מנוע לערכים מספריים

* נסדר את השמות הלא תואמים בעמודות של מודל ויצרן

* (נמחוק עמודות עם הרבה ערכים חסרים או עמודות שלא משפיעות על המודל- (לפי מה שראינו בבדיקות חוזרות של תוצאות המודל 

* נשתמש בפייפיליין על מנת להכין את הנתונים להכנסה למודל לימוד

* נשלב את הפעולות השונות לפי סוגי העמודות: מספריות וקטגוריאליות

* לבסוף נכין את נתוני בקלט והיעד

* לאחר מכן ביצענו בדיקת פיצרים על ידי בדיקת קרולציה בין הפיצרים לעמודת המחיר
* בבדיקת פיצרים בחרנו במדד שפיצר שלא עומד בו לא נכנס לדאטה  המוכן



In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def prepare_data(df):


    # Replace commas and handle non-numeric values for 'Km' and 'capacity_Engine' 
    def convert_to_float(value):
        try:
            return float(value.replace(',', ''))
        except:
            return np.nan
    
    # Apply the conversion function to 'Km' and 'capacity_Engine'
    df['Km'] = df['Km'].apply(convert_to_float)
    df['capacity_Engine'] = df['capacity_Engine'].apply(convert_to_float)
    
    # Fixing manufacturer and Engine_type names
    df['manufactor'] = df['manufactor'].replace('Lexsus', 'לקסוס')
    df['Engine_type']= df['Engine_type'].replace('היברידי', 'היבריד')

     # Drop columns with excessive missing values or less efect on the modal
    df.drop(columns=['Prev_ownership', 'Curr_ownership', 'Test', 'Pic_num', 'Cre_date', 'Repub_date', 'Description'], inplace=True)

    # Define numerical columns
    numerical_columns = ['Year', 'Hand', 'Km', 'capacity_Engine', 'Supply_score',]

    # Define categorical columns
    categorical_columns = ['manufactor', 'Gear', 'Engine_type', 'model', 'Color','Area','City']

    # Create preprocessing pipeline for numerical and categorical features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    # Combine preprocessing steps using ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical_columns),
            ('cat', categorical_transformer, categorical_columns)
        ])

    # Prepare features and target variable
    X = df.drop(columns=['Price'])
    y = df['Price']

    # Transform the data
    X_prepared = preprocessor.fit_transform(X)

    # Get feature names for the transformed data
    num_features = numerical_columns
    cat_features = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_columns)
    all_features = np.concatenate([num_features, cat_features])

    # Create a DataFrame with the transformed data
    df_prepared = pd.DataFrame(X_prepared, columns=all_features)

     # Add the target variable 'Price' back to the DataFrame
    df_prepared['Price'] = y
    
    # Calculate correlation matrix, to check which columns to keep
    corr_matrix = df_prepared.corr()

    # Select columns with correlation higher than a threshold with 'Price'
    threshold = 0.005
    high_corr_columns = corr_matrix[abs(corr_matrix['Price']) > threshold].index

    # Filter all_features to keep only columns with high correlation to 'Price'
    df_prepared = df_prepared[high_corr_columns]
    

    return df_prepared


# Load data
df = pd.read_csv('dataset.csv')

# Prepare data
df_prepared = prepare_data(df)

print(df_prepared.shape)


: 

##  TRAINING MODEL

*  נפריד בין הפיצרים לבין עמודת המחיר
*  נחלק את הדאטה לקבוצת אימון ולקבוצת בדיקה 
*  ניצור דגם מודל ונכניס אליו את קבוצת האימון 
*  הערכה באמצעות הערכה חוזרת
*  נזהה את התכונות עם המקדמים הכי גבוהים ונזהה את הסימנים שלהם




In [61]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn.metrics import mean_squared_error
import numpy as np

# Separate features and target variable
X = df_prepared.drop(columns=['Price'])
y = df_prepared['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Initialize ElasticNetCV model
elasticnet_cv = ElasticNet(random_state=42)

# Fit the model
elasticnet_cv, X_train, y_train
elasticnet_cv.fit(X_train, y_train)

# Evaluate using cross-validation
cv_scores = cross_val_score(elasticnet_cv, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)
print(f'Cross-validation RMSE scores: {cv_rmse_scores}')
print(f'Mean RMSE: {cv_rmse_scores.mean()}')

# Identify top 5 features with highest coefficients
coef_abs = pd.Series(elasticnet_cv.coef_, index=X.columns).abs()
top_features = coef_abs.nlargest(5)
print(f'Top 5 features:\n{top_features}')

# Determine the sign of the coefficients (positive or negative impact)
top_features_sign = pd.Series(elasticnet_cv.coef_, index=X.columns)
print(f'Sign of coefficients for top features:\n{top_features_sign[top_features.index]}')


Cross-validation RMSE scores: [16149.68878589 17604.95974649 17747.93998978 15839.63153562
 18336.76202046 19225.13584564 16411.52238489 19747.06198034
 17700.30541462 19124.08513714]
Mean RMSE: 17788.709284087432
Top 5 features:
Year                 6821.999901
Km                   2741.809480
capacity_Engine      2034.523444
Hand                 1645.829771
manufactor_טויוטה    1399.185014
dtype: float64
Sign of coefficients for top features:
Year                 6821.999901
Km                  -2741.809480
capacity_Engine      2034.523444
Hand                -1645.829771
manufactor_טויוטה    1399.185014
dtype: float64


## חמשת המאפיינים הכי משפיעים הם

##### Top 5 features:
* Year                 6821.999901
* Km                   2741.809480
* capacity_Engine      2034.523444
* Hand                 1645.829771
* manufactor_טויוטה    1399.185014

##### לאחר מכן בדקנו מה הסימן של כל מאפיין

#### Sign of coefficients for top features:

* Year                 6821.999901
* Km                  -2741.809480
* capacity_Engine      2034.523444
* Hand                -1645.829771
* manufactor_טויוטה    1399.185014
#### כולם חיוביים חוץ ממאפיינים קילומטר ויד

##### Mean RMSE: 17788.709284087432